# **0.  Mount Drive & Khai báo đường dẫn**

In [68]:
# PATH
# Dataset Path
dataset_path = "E:/An toàn thông tin/Code/Dataset/"

# Dataset including attack-type labels and difficulty level
train_set_source_path = dataset_path + 'NSL-KDD/csv_label&score/KDDTrain+.csv'
test_set_source_path = dataset_path + 'NSL-KDD/csv_label&score/KDDTest+.csv'
# Dataset Preprocessed Path
dataset_preprocessed_path = dataset_path + "NSL-KDD Preprocessed/Preprocess Encode and Normalize"

#**1. Chuẩn bị**

##**1.1 Import thư viện**

In [69]:
# Import Lib
import pandas as pd
import numpy as np

##**1.2. Load dataset**

In [70]:
#Load Dataset
train_set = pd.read_csv(train_set_source_path)

# Make a copy of train, test set
trainSet4Process = pd.read_csv(train_set_source_path)
testSet4Process = pd.read_csv(test_set_source_path)

# **2. Xử lý dữ liệu**
Xử lý các trường non-number: protocol_type, service, flag, class

> **Tập train:**
* protocol_type: 3
* service: 70
* flag: 11

> **Tập test:**
* protocol_type: 3
* service: 64
* flag: 11

> ***==> Sử dụng danh sách giá trị ở tập train để encode (Sử dụng danh sách các giá trị unique trong mỗi feature ở tập Train).***

## **2.0. Khai báo một số biến toàn cục**

In [71]:
# Nonnumeric features: 2, 3, 4, 42
NONNUMBERIC_FEATURES = ['protocol_type', 'service', 'flag']

# Binary value features: 7, 12, 14, 20, 21, 22
BINARY_FEATURES = ['land', 'logged_in', 'root_shell', 'num_outbound_cmds', 'is_host_login', 'is_guest_login']

## **2.1 Preprocess data**

###**1. Encode data**
* Với index là chỉ số của text trong unique_list (services, flags, protocol_types)
* Class: 
 * Normal: 0
 * Annomaly: 1
* Ngoài ra các trường có giá trị dạng Binary (1,0) thì chuyển sang dạng object để không cần phải chuẩn hóa (chỉ chuẩn hóa các feature có giá trị là số)

###**2. Convert BINARY_FEATURES to object type**
* Sau khi tiền dữ lý dữ liệu là bước chuẩn hóa dữ liệu (Normalize data) cho các numberic feature. 
* Để bỏ qua việc chuẩn hóa các Binary Feature, ta chuyển chúng về dạng object

###**3. Del Score Feature**
Xóa feature score do không sử dụng

In [72]:
# Print non-numberic before preprocess
for feature in NONNUMBERIC_FEATURES:
    print("{}\n{}\n\n".format(feature, train_set[feature].unique()))

protocol_type
['tcp' 'udp' 'icmp']


service
['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001' 'aol' 'http_2784' 'tftp_u' 'harvest']


flag
['SF' 'S0' 'REJ' 'RSTR' 'SH' 'RSTO' 'S1' 'RSTOS0' 'S3' 'S2' 'OTH']




In [73]:
# Preprocessing
for dataset in [trainSet4Process, testSet4Process]:
    # Encode for Non-Numberic Features
    for feature in NONNUMBERIC_FEATURES:
        listUniqueValues = train_set[feature].unique()
        dataset[feature] = dataset[feature].replace(listUniqueValues, range(1, len(listUniqueValues) + 1)).astype("int")
    
    # Convert BINARY_FEATURES to object type
    for feature in BINARY_FEATURES:
        dataset[feature] = dataset[feature].astype("object")

    # Delete Score Feature
    del dataset['score']

In [74]:
# Print data after Preprocessing
for dataset in [trainSet4Process, testSet4Process]:
    print("{}\n{}\n".format(100*'=', dataset))
    for feature in NONNUMBERIC_FEATURES:
        print("{}\n{}\n\n".format(feature, dataset[feature].unique()))

        duration  protocol_type  service  flag  src_bytes  dst_bytes land  \
0              0              1        1     1        491          0    0   
1              0              2        2     1        146          0    0   
2              0              1        3     2          0          0    0   
3              0              1        4     1        232       8153    0   
4              0              1        4     1        199        420    0   
...          ...            ...      ...   ...        ...        ...  ...   
125968         0              1        3     2          0          0    0   
125969         8              2        3     1        105        145    0   
125970         0              1       16     1       2231        384    0   
125971         0              1       36     2          0          0    0   
125972         0              1        1     1        151          0    0   

        wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0          

## **2.2 Normalize data**
Với các giá trị số, chuẩn hóa với hàm Min-Max chuyển giá trị về nằm trong khoảng [0,1]

In [75]:
# List numberic features
numbericFeatures = list(trainSet4Process.select_dtypes(include=['int',"float"]).columns)
print("Number of numberic features: {}".format(len(numbericFeatures)))
print(numbericFeatures)

Number of numberic features: 35
['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']


In [76]:
# Normalize with Min-Max
for dataset in [trainSet4Process, testSet4Process]:
    for numbericFeature in numbericFeatures:
        max_ = dataset[numbericFeature].max()
        min_ = dataset[numbericFeature].min()
        if (max_ != min_):
            dataset[numbericFeature] = dataset[numbericFeature].map(lambda x : (x - min_)/(max_ - min_))
        else:
            print(numbericFeature)

In [77]:
for dataset in [trainSet4Process, testSet4Process]:
  print("{}\n{}\n".format(100*'=', dataset))

        duration  protocol_type   service  flag     src_bytes     dst_bytes  \
0       0.000000            0.0  0.000000   0.0  3.558064e-07  0.000000e+00   
1       0.000000            0.5  0.014493   0.0  1.057999e-07  0.000000e+00   
2       0.000000            0.0  0.028986   0.1  0.000000e+00  0.000000e+00   
3       0.000000            0.0  0.043478   0.0  1.681203e-07  6.223962e-06   
4       0.000000            0.0  0.043478   0.0  1.442067e-07  3.206260e-07   
...          ...            ...       ...   ...           ...           ...   
125968  0.000000            0.0  0.028986   0.1  0.000000e+00  0.000000e+00   
125969  0.000186            0.5  0.028986   0.0  7.608895e-08  1.106923e-07   
125970  0.000000            0.0  0.217391   0.0  1.616709e-06  2.931438e-07   
125971  0.000000            0.0  0.507246   0.1  0.000000e+00  0.000000e+00   
125972  0.000000            0.0  0.000000   0.0  1.094232e-07  0.000000e+00   

       land  wrong_fragment  urgent  hot  ...  dst_

#**3. Lưu dataset**

In [78]:
# Save Dataset
trainSet4Process.to_csv(dataset_preprocessed_path+'KDDTrain+.csv', index=False)
testSet4Process.to_csv(dataset_preprocessed_path+'KDDTest+.csv', index=False)